In [2]:
# Importa a classe Word2Vec da biblioteca gensim, utilizada para criar e treinar modelos de embeddings de palavras com o algoritmo Word2Vec.
from gensim.models import Word2Vec

# Importa a classe CallbackAny2Vec da biblioteca gensim, que serve como base para criar callbacks personalizados durante o treinamento de modelos Word2Vec ou similares.
from gensim.models.callbacks import CallbackAny2Vec

import pandas as pd
from tqdm import tqdm
import numpy as np
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.utils as ku
from sklearn.model_selection import train_test_split



In [3]:
# Define uma classe de callback personalizada que herda de CallbackAny2Vec.
class SimpleTextCallback(CallbackAny2Vec):
    def __init__(self, total_epochs):
        self.epoch = 0  # Inicializa o contador de épocas (iterações de treinamento).
        self.total_epochs = total_epochs  # Armazena o número total de épocas para exibição.

    def on_epoch_end(self, model):
        self.epoch += 1  # Incrementa o contador de épocas ao final de cada época.
        print(f"Época {self.epoch} de {self.total_epochs}...")  # Imprime uma mensagem indicando a época atual e o total de épocas.

In [4]:
# Carrega dataframe salvo em formato pickle
df = pd.read_pickle("df_pre_processado.pkl")

# Converte a coluna 'Texto' do DataFrame em uma lista e armazena na variável preprocessed_articles, para ser usada no word2vec
preprocessed_articles = df['Texto'].tolist()

In [5]:
# Instancia o modelo Word2Vec com os seguintes parâmetros:
# vector_size=100: Dimensão do vetor de palavras gerado.
# window=5: Tamanho da janela de contexto ao redor de cada palavra.
# min_count=1: Ignora palavras com frequência total menor que 1.
# workers=4: Número de threads a serem usadas para treinar o modelo.
# Nota: Adicionar sg=0 altera o algoritmo para CBOW (Continuous Bag of Words).
model_word2vec = Word2Vec(vector_size=100, window=5, min_count=1, workers=4)

# Constrói o vocabulário do modelo com base nos artigos pré-processados.
model_word2vec.build_vocab(preprocessed_articles)

# Define o número total de épocas para o treinamento do modelo.
total_epochs = 10

# Instancia o callback que exibe o progresso do treinamento após cada época.
callback = SimpleTextCallback(total_epochs)

# Treina o modelo Word2Vec nos artigos pré-processados, utilizando o número total de exemplos e épocas definidos.
model_word2vec.train(preprocessed_articles, total_examples=model_word2vec.corpus_count, 
                     epochs=total_epochs, callbacks=[callback])

Época 1 de 10...
Época 2 de 10...
Época 3 de 10...
Época 4 de 10...
Época 5 de 10...
Época 6 de 10...
Época 7 de 10...
Época 8 de 10...
Época 9 de 10...
Época 10 de 10...


(36699385, 37650100)

In [6]:
# Salva o modelo Word2Vec para um arquivo
model_word2vec.save("model_word2vec.model")

In [7]:
# Encontra palavras mais similares de acordo com a palavra alvo
model_word2vec.wv.most_similar('governo')

[('governar', 0.6748394966125488),
 ('Governo', 0.5884649157524109),
 ('gestoes', 0.5602782368659973),
 ('aliado', 0.5227554440498352),
 ('gestao', 0.5091262459754944),
 ('economica', 0.4790595471858978),
 ('intervencao', 0.46907633543014526),
 ('governador', 0.4503372311592102),
 ('plano', 0.44774729013442993),
 ('concessoes', 0.44446662068367004)]

In [8]:
# Cria dois dicionários para mapear as palavras aos seus respectivos índices e vice-versa.
# Isso é feito para facilitar a conversão entre palavras e índices durante a criação de sequências
# numéricas que representam os textos. Essas sequências numéricas serão usadas como entrada para
# modelos de aprendizado de máquina, como redes neurais.
# Exemplo: 
# Suponha que o vocabulário seja ['casa', 'carro', 'jardim'], então os dicionários serão:
# word_to_index = {'casa': 1, 'carro': 2, 'jardim': 3}
# index_to_word = {1: 'casa', 2: 'carro', 3: 'jardim'}

# 'word_to_index' é um dicionário que mapeia cada palavra ao seu índice correspondente.
word_to_index = {}

# 'index_to_word' é um dicionário que mapeia cada índice à palavra correspondente.
index_to_word = {}

# Itera sobre a lista de palavras únicas obtida do modelo Word2Vec
for i, word in enumerate(model_word2vec.wv.index_to_key):
    # Atribui a palavra ao índice i + 1 no dicionário 'word_to_index'.
    # Os índices começam em 1 para reservar o índice 0 para preenchimento (padding) quando necessário.
    word_to_index[word] = i + 1
    
    # Atribui o índice i + 1 à palavra no dicionário 'index_to_word'.
    index_to_word[i + 1] = word

In [9]:
# 'sequences' é uma lista que armazenará as sequências numéricas correspondentes aos artigos pré-processados.
sequences = []

# Itera sobre os artigos pré-processados.
for tokens in tqdm(preprocessed_articles):
    # 'sequence' é uma lista temporária que armazenará a sequência numérica para o artigo atual.
    sequence = []
    
    # Itera sobre os tokens (palavras) no artigo atual.
    for token in tokens:
        # Verifica se o token atual está presente no dicionário 'word_to_index'.
        if token in word_to_index:
            # Se o token estiver presente, adiciona o índice correspondente à lista 'sequence'.
            sequence.append(word_to_index[token])
    
    # Após processar todos os tokens do artigo atual, adiciona a sequência numérica completa à lista 'sequences'.
    sequences.append(sequence)

  0%|          | 0/22284 [00:00<?, ?it/s]

100%|██████████| 22284/22284 [00:01<00:00, 17888.65it/s]


In [10]:
# Utiliza uma compreensão de lista para converter a sequência numérica do primeiro artigo em uma lista de palavras.
# Para cada índice 'i' na sequência numérica 'sequences[0]', obtém a palavra correspondente no dicionário 'index_to_word'.
# A compreensão de lista retorna uma lista de palavras.
word_list = [index_to_word[i] for i in sequences[0]]

# Usa o método 'join()' para combinar as palavras da lista 'word_list' em uma única string.
# As palavras são separadas por um espaço em branco.
text = " ".join(word_list)

# Imprime o texto reconstruído a partir da sequência numérica.
print(text)

poder decidir expulsar deputado federal Carlos gaguim partido Apos policia federal busca apreensoes gabinete de ele Camara legenda abrir espaco receber senadora expulsar pmdb katia abreu nota legenda informar afastamento parlamentar ja acordar filiado sigla “ parlamentar ter comunicar conclusao desfiliacao semana fato noticiar hoje executiva nacional poder solicitar imediato cancelamento filiacao quadro partido ” partido passado chegar cogitar lancar parlamentar candidato senado “ investigacao amplo apuracao eventual crime cometir consequente responsabilizacao envolvido ser puner maximo rigor lei independentemente posicao cargo ocupar ”


In [11]:
# Calcula o comprimento de cada sequência em 'sequences' e armazena os resultados na lista 'sequence_lengths'
sequence_lengths = [len(seq) for seq in sequences]

# Calcula a média dos comprimentos das sequências usando a função 'mean' do numpy
mean_length = np.mean(sequence_lengths)

# Obtém o comprimento máximo das sequências usando a função 'max' do Python
max_length = max(sequence_lengths)

# Imprime a média e o comprimento máximo das sequências
print("Mean sequence length: ", mean_length)
print("Maximum sequence length: ", max_length)

# Salva o valor de max_length em um arquivo pickle
with open('max_length.pkl', 'wb') as f:
    pickle.dump(max_length, f)

Mean sequence length:  168.95575300664154
Maximum sequence length:  3835


In [12]:
# Calcula 95% dos comprimentos das sequências (preserva 95% das sequências)
max_length = int(np.percentile(sequence_lengths, 95))

# Realiza o preenchimento das sequências com base no novo 'max_length'
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

In [13]:
df['label'].nunique() # Retorna o número de categorias únicas (classes) presentes na coluna 'label'

2

In [14]:
# Converte rótulos para one-hot encoding
num_classes = df['label'].nunique()
labels = ku.to_categorical(df['label'], num_classes)

In [15]:
labels

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [16]:
# Salva os rótulos (labels) em um arquivo pickle
with open('labels.pkl', 'wb') as f:
    pickle.dump(labels, f)

In [17]:
# Divide os dados em um conjunto de treino (70%) e um conjunto temporário (30%)
X_train, X_temp, y_train, y_temp = train_test_split(padded_sequences, labels, test_size=0.3, random_state=42)

# Divide o conjunto temporário em conjuntos de validação (15%) e teste (15%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  

In [18]:
# Salva os dados divididos em arquivos pickle
with open('train_data.pkl', 'wb') as f:
    pickle.dump((X_train, y_train), f)

with open('val_data.pkl', 'wb') as f:
    pickle.dump((X_val, y_val), f)

with open('test_data.pkl', 'wb') as f:
    pickle.dump((X_test, y_test), f)